In [71]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity


load_dotenv()

True

In [ ]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL")
)

model_name = os.getenv("MODEL_NAME")

In [25]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

# Get LLM Response

In [ ]:
messages = [{"role": "user", "content": "What is the capital of mongolia?"}]
response = get_chatbot_response(client, model_name, messages)

In [27]:
response

'The capital of Mongolia is Ulaanbaatar (also known as Ulan Bator).'

# Prompt Engineering

## Structured output

In [ ]:
system_prompt = """
You are a helpful assistant that answers questions about capitals of countries.

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than the JSON object:

[
{
    "country": the country that you will get the capital of
    "capital": the capital of the country stated
}

]

"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What is the capital of mongolia?"})
response = get_chatbot_response(client, model_name, messages)

'[\n{\n    "country": "Mongolia",\n    "capital": "Ulaanbaatar"\n}\n]'

In [35]:
json_response = json.loads(response)
type(json_response[0])

dict

In [34]:
print(json_response)

[{'country': 'Mongolia', 'capital': 'Ulaanbaatar'}]


## Input structuring

In [36]:
user_input = """
Give me the capitals of the following countries:
```
1. Spain
2. America
3. Iran
```
"""

In [37]:
messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)

In [38]:
print(response)

[
{
    "country": "Spain",
    "capital": "Madrid"
},
{
    "country": "America",
    "capital": "Washington D.C."
},
{
    "country": "Iran",
    "capital": "Tehran"
}
]


# Give the model time to think (Chain of thought)

In [39]:
user_prompt = """
Calculate the  result of the equation 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object.


{
    "result": The final number resulted from the calculating the equation above
}

"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)

In [40]:
print(json.loads(response))

{'result': 4}


In [ ]:
259/2*8654+91072*33-12971

4113098.0

In [45]:
user_prompt = """
Calculate the  result of the equation 259/2*8654+91072*33-12971


Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object.


{
    "steps": this is where you solve the equation bit by bit following the BEDMAS order of operations and you need to show your work and calculate each step leading to the final result and feel free to write in free text
    "result": The final number resulted from the calculating the equation above
}

"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)

In [43]:
print(response)

{
    "result": 1343519
}


In [47]:
4113098 - 1343519

2769579

In [ ]:
print(response)

{
    "steps": "First, we need to follow the order of operations (BEDMAS):

1. Divide 259 by 2: 259 ÷ 2 = 129.5
2. Multiply 129.5 by 8654: 129.5 × 8654 = 112,141.3
3. Multiply 91072 by 33: 91072 × 33 = 3,003,336
4. Subtract 12971 from 3,003,336: 3,003,336 - 12,971 = 2,990,365
5. Add 112,141.3 to 2,990,365: 2,990,365 + 112,141.3 = 3,102,506.3

",
    "result": 3102506.3
}


In [48]:
4113098 - 3102506.3

1010591.7000000002

In [50]:
2769579/1010591.7

2.7405518964780735

# Retrieval Augmented Generation

In [ ]:
user_prompt = '''
What's new in iphone 16?
'''

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)

In [52]:
print(response)

There is no such thing as an "iPhone 16". Apple has not released an iPhone with that model number. The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Mini, iPhone 13 Pro, and iPhone 13 Pro Max.

However, if you're looking for information on the latest rumors and leaks about the next-generation iPhone, I can provide you with some information.

Rumors suggest that the next-generation iPhone, likely to be called iPhone 14, will feature several upgrades and improvements, including:

1. Improved cameras: The iPhone 14 is expected to feature a new triple-camera setup with a wider-angle lens, a telephoto lens, and a ultra-wide-angle lens.
2. Faster processor: The iPhone 14 is expected to be powered by Apple's A15 Bionic chip, which will provide a significant boost in performance and efficiency.
3. Longer battery life: The iPhone 14 is expected to have a longer battery life, with some reports suggesting up to 12 hours of internet use on a

In [54]:
iphone_16 = """
CUPERTINO, CALIFORNIA Apple today announced iPhone 16 and iPhone 16 Plus, built for Apple Intelligence, the easy-to-use personal intelligence system that understands personal context to deliver intelligence that is helpful and relevant while protecting user privacy. The iPhone 16 lineup also introduces Camera Control, which brings new ways to capture memories, and will help users quickly access visual intelligence to learn about objects or places around them faster than ever before. The powerful camera system features a 48MP Fusion camera with a 2x Telephoto option, giving users two cameras in one, while a new Ultra Wide camera enables macro photography. Next-generation Photographic Styles help users personalize their images, and spatial photo and video capture allows users to relive life’s precious memories with remarkable depth on Apple Vision Pro. The new A18 chip delivers a huge leap in performance and efficiency, enabling demanding AAA games, as well as a big boost in battery life.
iPhone 16 and iPhone 16 Plus will be available in five bold colors: black, white, pink, teal, and ultramarine. Pre-orders begin Friday, September 13, with availability beginning Friday, September 20.
“iPhone 16 and iPhone 16 Plus mark the beginning of a new era for iPhone with Apple Intelligence delivering powerful, personal, and private experiences to our users,” said Kaiann Drance, Apple’s vice president of Worldwide iPhone Product Marketing. “With new ways to discover the world around you and capture memories using Camera Control; a 48MP Fusion camera that gives you two optical-quality cameras in one; a big boost in battery life; and powerful, efficient performance thanks to the A18 chip, this is the perfect time for customers to upgrade or make the switch to iPhone.”
Apple Intelligence unlocks new ways for users to enhance their writing. With systemwide Writing Tools built into iOS 18, users can rewrite, proofread, and summarize text nearly everywhere they write, including Mail, Notes, Pages, and third-party apps. In the Notes and Phone apps, users can also record, transcribe, and summarize audio. When a recording is initiated while on a call in the Phone app, participants are automatically notified, and once the call ends, Apple Intelligence generates a summary to recall key points.
"""

In [55]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16

"""
messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

According to the article, the new features and improvements in iPhone 16 and iPhone 16 Plus include:

1. Apple Intelligence: a personal intelligence system that understands personal context to deliver helpful and relevant information while protecting user privacy.
2. Camera Control: new ways to capture memories, including:
	* 48MP Fusion camera with a 2x Telephoto option, giving users two cameras in one.
	* Ultra Wide camera for macro photography.
	* Next-generation Photographic Styles to personalize images.
	* Spatial photo and video capture for remarkable depth on Apple Vision Pro.
3. A18 chip: delivers a huge leap in performance and efficiency, enabling demanding AAA games and a big boost in battery life.
4. New colors: available in five bold colors: black, white, pink, teal, and ultramarine.
5. Apple Intelligence Writing Tools: systemwide tools built into iOS 18 that allow users to rewrite, proofread, and summarize text nearly everywhere they write, including Mail, Notes, Pages, an

### Automatically extract context data from database

In [57]:
samsung_s24 = """
The Galaxy S24 Series is here, and with it comes the launch of Galaxy AI, which is set to take your mobile experience to the next level!

Samsung’s latest Galaxy S24 Series launch is packed with exciting new features that are sure to get you excited. The series, which includes the Galaxy S24, S24+ and S24 Ultra, is all about new innovations that are set to enrich your daily life.

So what can you expect from these latest phones? Let’s find out!

1. Language will no longer be a barrier for you!
Imagine being on a business trip in Korea and you need to communicate with your Korean client, or a family holiday in France and you want to book dinner at a popular restaurant – but, you don’t speak the local languages. Fret not, because with a Galaxy S24 Series phone in your hands, communicating with someone in a different language is no longer going to be an issue!

Live Translate
The Galaxy S24 Series makes communicating with someone who speaks a different language much easier! Live Translate1, which leverages Galaxy AI, allows you to enjoy real-time translation during your phone calls. Simply activate the Live Translate feature on your Galaxy S24 Series phone and you are all set to go! What do we mean?

Interpreter
Having a chat with locals on your travels will also no longer be an issue, because the Interpreter2 feature on the Galaxy S24 Series will be your personal translator! This built-in feature, which is set on a user-friendly split screen interface, allows you to communicate easily. You’ll just need to download the relevant language packs before you travel.

Chat Assist
Do you find it frustrating when you can’t convey the right tone in your messages? With Galaxy S24’s Chat Assist3, you can easily fine-tune your content, in just a few seconds, whether you want to have it as a polite message to your superior, or a catchy social media post! How convenient!

2. Easy note-taking for work & school

Transcript Assist
Do you often spend hours trying to transcribe meeting minutes or your professor’s lectures? With the Galaxy S24 Series, that will soon be a thing of the past because you can now use Transcript Assist4 to transcribe voice recordings that you take on the pre-installed Samsung Voice Recorder app!

Simply record during your lectures or meetings and use Transcript Assist for transcription. The feature is even able to detect multiple speakers!

After transcribing, you can also get your document translated or summarised! It doesn’t get easier than that.

3. A new way to search

Circle to Search with Google
Saw a top you like in an influencer's video and wondered where it's from? Or maybe, you want to know what breed that cute little puppy in the photo is!

Circle to Search with Google5 will give you the answers you seek in just seconds! All you’ve got to do is circle the object of interest with your finger or S-Pen, and the Google web search will begin. How amazing is it that your curiosities and questions can now be easily answered!

4. Capture photos, edit, and create videos like never before!
ProVisual Engine
Imagine this: You’re at a concert, far from the stage, but want to capture your favourite band's epic moments. With the Galaxy S24 Ultra, you now can do so without losing a speck of detail. The Quad Tele System6, featuring a new 5x optical zoom lens, works in harmony with the 50MP sensor. This means, you can enjoy optical-quality shots with zoom levels from 2x, 3x, 5x to 10x magnification, all thanks to the Adaptive Pixel Sensor.

If you want to score a close-up shot of the lead singer, but your seat is all the way to the back, fret not, because with the deep learning based AI Zoom7, you can seamlessly go up to 100x zoom, and ensure that your images are not just close-up, but crystal clear! No more shaky, pixelated close up shots – just vivid, sharp pictures that will make you feel like you are right there in the front row!

Nightography
With the Galaxy S24 Series, night shots and videos will be your new playground. With the enhanced Nightography of the Galaxy S24 Ultra, you’ll be able to capture vibrant, detailed memories, even in dimly lit scenes8.

Trying to get those zoomed in club shots at your friend’s bachelorette party? Want to create a video of the beautiful night city skyline for your socials? That’s going to be really easy with the Galaxy S24 Ultra Space Zoom, which is brilliant in any conditions. With a larger 1.4 micrometre pixel size, which is a whopping 60% increase from the Galaxy S23 Ultra, you will be able to capture more light even in dim conditions.

But that’s not all! Because the Galaxy S24 Ultra also boasts a wider optical image stabiliser (OIS) and advanced hand-shake compensation, so you can say goodbye to blurry and shaky night shots. Just imagine the kind of pictures you can take with this phone!

And if you are a video content creator, or just love taking videos at night, then you would be glad to know that the Galaxy S24 Ultra is a game-changer as both the front and rear cameras are equipped with a Dedicated ISP Block that gives you superior noise reduction. To top it off, the Galaxy S24 Ultra is also able to analyse gyro information to differentiate between your movements and the subject’s. What does this mean for you? Stunningly clear videos that make you feel like you are reliving the moment, not just watching it!

Galaxy AI editing tools
Do you spend hours editing your pictures for your social media? Now, with the Galaxy AI editing tools that are on the Galaxy S24 Series, editing photos will just take you a few seconds! You can now easily remove objects or reflections, or even use Generative Edit to fill parts of your image background with Generative AI!

Battery
Despite its advanced AI features, the Galaxy S24 Series doesn't compromise on battery life. It achieves this balance through new chipsets with smart battery management and larger batteries in the Galaxy S24 and S24+ models. The Galaxy S24 Ultra boasts a 5000 mAh battery, while the Galaxy S24 and S24+ come with 4000mAh and 4900mAh batteries, respectively12. This means you can enjoy all the AI enhancements without constantly worrying about your phone's battery life!

"""

In [58]:
data = [iphone_16, samsung_s24]

user_prompt = """What's new in iphone 16"""

In [59]:
embedding_client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_EMBEDDING_URL")
)

In [62]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(input=text_input, model=model_name)

    embeddings = []
    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)
    
    return embeddings

In [63]:
embedding_model = "BAAI/bge-small-en-v1.5"

In [68]:
user_prompt_embedding = get_embedding(embedding_client, embedding_model, user_prompt)

In [78]:
data_embeddings = [get_embedding(embedding_client, embedding_model, text)[0] for text in data]

In [80]:
data_embeddings

[[-0.03812352567911148,
  -0.02623472549021244,
  -0.008271797560155392,
  0.002905430970713496,
  -0.041572652757167816,
  -0.0027547224890440702,
  0.03458283096551895,
  -0.004734917543828487,
  -0.011186767369508743,
  0.030050130560994148,
  0.05036334693431854,
  0.018832838162779808,
  -0.021198390051722527,
  -0.002195765497162938,
  0.060802292078733444,
  0.028050856664776802,
  0.07881100475788116,
  -0.18204060196876526,
  -0.04331247881054878,
  -0.012644251808524132,
  0.00483411829918623,
  -0.022938214242458344,
  -0.03934445604681969,
  -0.01919911801815033,
  -0.008660969324409962,
  0.036749981343746185,
  -0.0015814853832125664,
  -0.029592281207442284,
  -0.03864242136478424,
  -0.18606966733932495,
  0.001022528507746756,
  0.004078668076545,
  0.013269978575408459,
  -0.020572664216160774,
  -0.04142003506422043,
  -0.030874257907271385,
  -0.043678756803274155,
  0.02911917120218277,
  -0.004505993332713842,
  0.017672955989837646,
  0.011904063634574413,
  0.01

In [81]:
data_similarity = cosine_similarity(user_prompt_embedding, data_embeddings)

In [ ]:
data_similarity

array([[0.81037814, 0.5503224 ]])

In [84]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

np.int64(0)

In [85]:
data[closest_entry_index]

'\nCUPERTINO, CALIFORNIA Apple today announced iPhone 16 and iPhone 16 Plus, built for Apple Intelligence, the easy-to-use personal intelligence system that understands personal context to deliver intelligence that is helpful and relevant while protecting user privacy. The iPhone 16 lineup also introduces Camera Control, which brings new ways to capture memories, and will help users quickly access visual intelligence to learn about objects or places around them faster than ever before. The powerful camera system features a 48MP Fusion camera with a 2x Telephoto option, giving users two cameras in one, while a new Ultra Wide camera enables macro photography. Next-generation Photographic Styles help users personalize their images, and spatial photo and video capture allows users to relive life’s precious memories with remarkable depth on Apple Vision Pro. The new A18 chip delivers a huge leap in performance and efficiency, enabling demanding AAA games, as well as a big boost in battery l

In [86]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [87]:
print(user_prompt_with_data)



CUPERTINO, CALIFORNIA Apple today announced iPhone 16 and iPhone 16 Plus, built for Apple Intelligence, the easy-to-use personal intelligence system that understands personal context to deliver intelligence that is helpful and relevant while protecting user privacy. The iPhone 16 lineup also introduces Camera Control, which brings new ways to capture memories, and will help users quickly access visual intelligence to learn about objects or places around them faster than ever before. The powerful camera system features a 48MP Fusion camera with a 2x Telephoto option, giving users two cameras in one, while a new Ultra Wide camera enables macro photography. Next-generation Photographic Styles help users personalize their images, and spatial photo and video capture allows users to relive life’s precious memories with remarkable depth on Apple Vision Pro. The new A18 chip delivers a huge leap in performance and efficiency, enabling demanding AAA games, as well as a big boost in battery li

In [88]:
messages = [{"role": "user", "content": user_prompt_with_data}]
response = get_chatbot_response(client, model_name, messages)

In [89]:
print(response)

According to the article, the new features and improvements in iPhone 16 and iPhone 16 Plus include:

1. Apple Intelligence: a personal intelligence system that understands personal context to deliver helpful and relevant information while protecting user privacy.
2. Camera Control: new ways to capture memories, including:
	* 48MP Fusion camera with a 2x Telephoto option, giving users two cameras in one.
	* Ultra Wide camera for macro photography.
	* Next-generation Photographic Styles to personalize images.
	* Spatial photo and video capture for remarkable depth on Apple Vision Pro.
3. A18 chip: delivers a huge leap in performance and efficiency, enabling demanding AAA games and a big boost in battery life.
4. New colors: available in five bold colors: black, white, pink, teal, and ultramarine.
5. Apple Intelligence Writing Tools: systemwide tools built into iOS 18 that allow users to rewrite, proofread, and summarize text nearly everywhere they write, including Mail, Notes, Pages, an